In [29]:
import json
import re
import twint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('rslp')

%matplotlib inline

[nltk_data] Downloading package stopwords to /home/otavio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/otavio/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [13]:
sentiments = ['alegria', 'medo', 'tristeza', 'nojo', 'surpresa', 'raiva']
base_treinamento = []

for sentiment in sentiments:
    inputjson_filename = '{sentiment_filename}.json'.format(sentiment_filename = sentiment)
    outputjson_filename = '{sentiment_filename}_output.json'.format(sentiment_filename = sentiment)
    
    # Deserialize file to load json in data
    with open(inputjson_filename) as f:
        data = json.load(f)

    # Filter python objects with list comprehensions
    output_dict = [x for x in data if x['language'] == 'pt']

    # Transform python object back into json
    output_json = json.dumps(output_dict, ensure_ascii=False)

    # Show json
    #if isinstance(output_json, str):
     #   print(output_json)

    # Regex
    pattern = r'(?<="tweet": )"(.*?)"'

    # Get all regex matches
    tweets = re.findall(pattern, output_json)
    
    for tweet in tweets:
        base_treinamento.append((tweet, sentiment))
        
    print(base_treinamento)
    #tweets_tuplas = []

    # Create json object with tweet and sentiment
    #for tweet in tweets:
    #    tweets_tuplas.append({'tweet': tweet, 'sentiment': sentiment})

    # Open json file
    #out_file = open(outputjson_filename, "w")

    # Dump array into json file
    #json.dump(tweets_tuplas, out_file, indent = 6, ensure_ascii=False)

    # Close json file
    #out_file.close()
    


[('Melhor de tudo é que quarta-feira o jogo é contra o atlético mg e não temos zagueiros kkkkkkk que alegria', 'alegria'), ('Ela me enche de alegria e orgulho 🥺🥰', 'alegria'), ('e a minha alegria foi p casa do crlh, que ódio', 'alegria'), ('A alegria do Galhardo, tendo que sair de novo depois de expulsão de zagueiro kkkkkk 🤦🏻\u200d♀️', 'alegria'), ('Já chorei com a minha família hoje é só alegria 💪🏼😭', 'alegria'), ('@neymarjr Slc irmão sdds do tempo que nós jogamos juntos, só alegria do seu lado🥺', 'alegria'), ('O promotor falando pro Akgün que eles vão morar juntos novamente e alegria do meu filho 🤧', 'alegria'), ('@augustodeAB @jdoriajr Sou de SP e essa felicidade é alegria de pobre. Alagoas tá na frente, Maranhão, Ceará, Santa Catarina,... não tá com essa bola toda não!', 'alegria'), ('@ellenzinhakkk obrigado viu tava ótimo já agora tô pulando de alegria', 'alegria'), ('@vinniciusrz infância alegria felicidade ..😓', 'alegria'), ('@zapataedicoes @EricPeleias Venho reforçar o pedido, 

In [14]:
exemplo_base = pd.DataFrame(base_treinamento)
exemplo_base.columns = ['Frase', 'Sentimento']

In [16]:
print('Tamanho da base de Treinamento {}'.format(exemplo_base.shape[0]))
exemplo_base.Sentimento.value_counts()

Tamanho da base de Treinamento 968
surpresa    20.557851
raiva       20.351240
medo        19.318182
nojo        15.909091
tristeza    13.636364
alegria     10.227273
Name: Sentimento, dtype: float64


In [ ]:
print((exemplo_base.Sentimento.value_counts() / exemplo_base.shape[0]) * 100)

In [18]:
exemplo_base.sample(n=20)

,Frase,Sentimento
109,@igorfernando72 teu amigo pedrin cheio de medo...,medo
620,@povddl Não é de hj que ele pega carona no suc...,surpresa
732,"@r_annn3 kkkkkkkk cedo dms tu, ome porém tava...",surpresa
760,eu ganhei uma FESTA SURPRESA,surpresa
964,@DCM_online Eu to com mt raiva dele nossa. Sér...,raiva
390,Que tristeza,tristeza
404,"lidando com a tristeza de um jeito saudável, v...",tristeza
659,@fqt299 @LGAndriolo Fui olhar seu perfil e nen...,surpresa
39,"minha mãe fez a graça, pagou a pizza, meu filh...",alegria
844,ele me deu uma pulseira e eu comi o presente d...,raiva


In [20]:
base_teste = [('que merda vai tomar no cu', 'raiva'),
              ('hoje foi só decepção', 'tristeza'),
              ('mano que susto, não durmo hoje', 'medo'),
              ('aeeeee finalmente consegui! uhu', 'alegria'),
              ('caralho eu ganhei um ps5!', 'surpresa'),
              ('eca o pombo cagou aqui', 'nojo')
             ]

In [22]:
exemplo_base_teste = pd.DataFrame(base_teste)
exemplo_base_teste.columns = ['Frase', 'Sentimento']
print('Tamanho da base de Teste{}'.format(exemplo_base_teste.shape[0]))
exemplo_base_teste.Sentimento.value_counts()

Tamanho da base de Teste6


tristeza    1
nojo        1
alegria     1
medo        1
raiva       1
surpresa    1
Name: Sentimento, dtype: int64

In [25]:
lista_Stop = nltk.corpus.stopwords.words('portuguese')
np.transpose(lista_Stop)

array(['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para',
       'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as',
       'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou',
       'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo',
       'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo',
       'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa',
       'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos',
       'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas',
       'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus',
       'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos',
       'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele',
       'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está',
       'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram',
       'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos

In [26]:
def removeStopWords(texto):
    frases = []
    for(palavras, sentimento) in texto:
        # Criamos uma list compreheension para extrair apenas as palvaras que não estão na lista_Stop
        semStop = [p for p in palavras.split() if p not in lista_Stop]
        #Inserindo as frases com os labels(sentimento) já tratados pela lista_Stop
        frases.append((semStop, sentimento))
    return frases

In [27]:
def aplica_Stemmer(texto):
    stemmer = nltk.stem.RSLPStemmer()
    # Escolhido o RSLPS pois é especifico da lingua portugesa
    frases_sem_Stemming = []
    for(palavras, sentimento) in texto:
        com_Stemming = [str(stemmer.stem(p)) for p in palavras.split() if p not in lista_Stop]
        frases_sem_Stemming.append((com_Stemming, sentimento))
    return frases_sem_Stemming

In [30]:
frases_com_Stem_treinamento = aplica_Stemmer(base_treinamento)

In [31]:
pd.DataFrame(frases_com_Stem_treinamento, columns=['Frase', 'Sentimento']).sample(10)

,Frase,Sentimento
235,"[perceb, tempo,, med, fal, ser, cancel]",medo
937,"[@ariela_cid, @ld_cml, @ceduyang, se, depress,...",raiva
159,"[nao, sei, pq, pedr, tol, tant, brux, kra....,...",medo
929,"[@luzlucas_, desd, pênalt, inexist, contr, pal...",raiva
229,"[@rbooks, amg,, med, bich, sair, ral, ach, chi...",medo
460,"[eu, tav, trist, est, perd, agor, tá, deix, no...",nojo
858,"[só, raiv, pprt]",raiva
942,"[@louelight, -, apelido:, bich, -, gost, você?...",raiva
221,"[quer, assum, pra, ver, oq, ia, falar,, porém,...",medo
746,"[até, agor, apaixon, surpr, júni, fez, pra, mi...",surpresa


In [33]:
frases_com_Stem_teste = aplica_Stemmer(base_teste)

In [34]:
def busca_Palavras(frases):
    todas_Palavras = []
    for(palavras, sentimento) in frases:
        todas_Palavras.extend(palavras)
    return todas_Palavras

In [35]:
palavras_treinamento = busca_Palavras(frases_com_Stem_treinamento)
palavras_teste = busca_Palavras(frases_com_Stem_teste)

In [36]:
print("Quantidade de palavaras na base de Treinamento {}".format(pd.DataFrame(palavras_treinamento).count()))

Quantidade de palavaras na base de Treinamento 0    10806
dtype: int64


In [37]:
def busca_frequencia(palavras):
    palavras = nltk.FreqDist(palavras)
    return palavras

In [38]:
frequencia_treinamento = busca_frequencia(palavras_treinamento)

In [39]:
frequencia_treinamento.most_common(20)

[('med', 175),
 ('surpr', 166),
 ('•', 152),
 ('raiv', 147),
 ('noj', 133),
 ('q', 128),
 ('trist', 123),
 ('pra', 119),
 ('vc', 85),
 ('alegr', 82),
 ('fic', 77),
 ('gost', 68),
 ('faz', 66),
 ('eu', 62),
 ('que', 58),
 ('tô', 57),
 ('dia', 55),
 ('\\', 55),
 ('pq', 53),
 ('ser', 52)]

In [40]:
frequencia_teste = busca_frequencia(palavras_teste)

In [41]:
frequencia_teste.most_common(20)

[('hoj', 2),
 ('merd', 1),
 ('vai', 1),
 ('tom', 1),
 ('cu', 1),
 ('decepç', 1),
 ('man', 1),
 ('susto,', 1),
 ('durm', 1),
 ('aeeee', 1),
 ('final', 1),
 ('consegui!', 1),
 ('uhu', 1),
 ('caralh', 1),
 ('ganh', 1),
 ('ps5!', 1),
 ('eca', 1),
 ('pomb', 1),
 ('cag', 1),
 ('aqu', 1)]

In [42]:
def busca_palavras_unicas(frequencia):
    freq = frequencia.keys()
    return freq

palavras_unicas_treinamento = busca_palavras_unicas(frequencia_treinamento)
palavras_unicas_teste = busca_palavras_unicas(frequencia_teste)

In [43]:
def extrator_palavras(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_treinamento:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [44]:
def extrator_palavras_teste(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_teste:
        caracteristicas['%s' % palavras] = (palavras in doc)
    return caracteristicas

In [45]:
base_completa_treinamento = nltk.classify.apply_features(extrator_palavras, frases_com_Stem_treinamento)
base_completa_teste = nltk.classify.apply_features(extrator_palavras_teste, frases_com_Stem_teste)

In [46]:
classificador = nltk.NaiveBayesClassifier.train(base_completa_treinamento)
print(classificador.labels())

['alegria', 'medo', 'tristeza', 'nojo', 'surpresa', 'raiva']


In [47]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                   trist = True           triste : medo   =    103.2 : 1.0
                   alegr = True           alegri : medo   =     98.4 : 1.0
                     noj = True             nojo : triste =     75.2 : 1.0
                     med = True             medo : alegri =     59.4 : 1.0
                     fiq = True            raiva : medo   =     23.1 : 1.0
                    gost = True            raiva : triste =     15.9 : 1.0
                       • = True            raiva : surpre =     13.1 : 1.0
                       \ = True            raiva : triste =     12.3 : 1.0
                    sint = True            raiva : triste =     11.9 : 1.0
                     hoj = True           alegri : surpre =     11.3 : 1.0
None


In [48]:
print(nltk.classify.accuracy(classificador, base_completa_teste))

0.3333333333333333
